In [1]:
import os
import yaml
import pickle
import numpy as np
import pandas as pd
import itertools
import scipy
import torch
import torch_geometric.datasets as datasets
import torch_geometric.data as data
import torch_geometric.transforms as transforms
import networkx as nx
from torch_geometric.utils.convert import to_networkx
import matplotlib.pyplot as plt

from produce_dataset import *

In [2]:
ntuple_dir = '/ecoderemdvol/EleGun/EPGun-PU200/data/econ_ntuples/0001/'
root_dir = 'FloatingpointAutoEncoderEMDAEMSEttbarDummyHistomaxGenmatchGenclustersntuple'

df_econ = loadEconData(ntuple_dir,root_dir,'econ_data.csv',False)

df_gen = loadGenData(ntuple_dir,root_dir,'gen_data.csv')

In [3]:
def build_graph(df_econ,
            df_gen,
            zside_select,
            phi_min,
            phi_max):
    df_econ = df_econ.astype('double')
    df_econ_front_endcap = df_econ[df_econ.zside == zside_select]
    df_nodes=df_econ_front_endcap[['ECON_0', 'ECON_1', 'ECON_2', 'ECON_3', 'ECON_4', 'ECON_5', 'ECON_6',
           'ECON_7', 'ECON_8', 'ECON_9', 'ECON_10', 'ECON_11', 'ECON_12','ECON_13', 'ECON_14', 'ECON_15', 
            'wafer_energy', 'tc_eta', 'tc_phi']]
    df_nodes.reset_index(inplace=True)

    #only choose wafers in some range now
    #know electron is ~ -pi/2 and choose 30 degree range around it 1.31 to 1.93

    df_econ_front_endcap_30deg = df_econ_front_endcap[df_econ_front_endcap.tc_phi >= phi_min]
    df_econ_front_endcap_30deg = df_econ_front_endcap_30deg[df_econ_front_endcap_30deg.tc_phi <=phi_max]

    df_econ_front_endcap_30deg.reset_index(inplace=True)
    df_nodes=df_econ_front_endcap_30deg
    embeddings = torch.tensor(df_nodes.values)

    eta=df_nodes['tc_eta']
    phi=df_nodes['tc_phi']
    idx = range(len(eta))
    indices_i = np.array([i for i,j in itertools.product(idx,idx)])
    indices_j = np.array([j for i,j in itertools.product(idx,idx)])

    del_R = np.empty([len(eta),len(eta)])
    del_R.shape

    for (i, j) in zip(indices_i,indices_j):
        del_R[i][j]=np.sqrt((eta[i]-eta[j])**2+((phi[i]-phi[j])%(2*np.pi))**2)

    del_R = torch.tensor(del_R)

    adj = np.zeros([len(eta),len(eta)])
    for (i, j) in zip(indices_i,indices_j):
        if del_R[i][j] <0.045 and  del_R[i][j]> 0 :
            adj[i][j]=1
    adj = adj.astype('double')
    adj=torch.tensor(adj) 

    edge_index = (adj > 0).nonzero().t()
    edge_index.shape

    #give each graph electron pt to predict

    feature_df= (df_gen[df_gen.phi>phi_min][df_gen.phi<phi_max])
    features = torch.tensor(feature_df.values)

    graph = data.Data(x=embeddings, edge_index=edge_index, y=features)
    
    return graph

In [4]:
df_gen

,pt,energy,eta,phi
0,80.423019,522.664307,2.558814,-1.537044
1,80.423019,522.664307,-2.558814,1.604549


In [5]:
graph_1 = build_graph(df_econ,df_gen,1,-1.93,-1.3)

In [6]:
graph_1

Data(x=[667, 25], edge_index=[2, 2498], y=[1, 4])

In [ ]:
graph_2 = graph_1 = build_graph(df_econ,df_gen,-1,1.60-(np.pi/6),1.60+(np.pi/6))

In [ ]:
graph_2

In [7]:
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

In [8]:
dataset=graph_1
class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 4)
        self.conv2 = GCNConv(16,2)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1).to(torch.float)

In [9]:
model = GCN()
data = graph_1
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

RuntimeError: expected scalar type Double but found Float